[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/carloszan/bert-fine-tuning-tutorial/blob/main/create-model.ipynb)

In [ ]:
!pip install transformers
!pip install datasets

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

dataset = load_dataset('ic-fspml/fpb')

/home/carlos/Documents/git/bert-tutorial-financial-analysis/.env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

mapDict = {
    "positive": 0,
    "neutral": 1,
    "negative": 2
}

def transform_labels(label):
    label = label["label"]

    result = []
    for l in label:
      result.append(mapDict[l])


    return {"label": result} 

def tokenize_function(example):
    return tokenizer(example["sentence"], padding=True, truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.map(transform_labels, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [5]:
from transformers import TrainingArguments

output_dir = "./bert-financial-sentiment-analysis"
training_args = TrainingArguments(
  output_dir=output_dir,
  num_train_epochs=3,  # Number of training epochs
  per_device_train_batch_size=8,  # Batch size per GPU
  per_device_eval_batch_size=8,   # Batch size for evaluation per GPU
  weight_decay=0.01,   # Strength of weight decay
  logging_dir="./logs",   # Directory for storing logs
  logging_steps=100,   # Log every N steps
  evaluation_strategy="steps",   # Evaluation strategy during training
  eval_steps=200,   # Run evaluation every N steps
  save_total_limit=2,   # Only save the last N checkpoints
  save_steps=200,   # Save checkpoint every N steps
  load_best_model_at_end=True,   # Load the best model at the end of training
  metric_for_best_model="accuracy",   # Metric to use for the best model
)


In [6]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import Trainer
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [8]:
trainer.train()

  7%|▋         | 101/1455 [00:17<03:36,  6.26it/s]

{'loss': 0.7072, 'learning_rate': 4.656357388316151e-05, 'epoch': 0.21}


 14%|█▎        | 200/1455 [00:33<03:18,  6.32it/s]

{'loss': 0.5398, 'learning_rate': 4.312714776632303e-05, 'epoch': 0.41}


                                                  
 14%|█▎        | 200/1455 [00:37<03:18,  6.32it/s]

...

                                                   
 96%|█████████▌| 1400/1455 [04:38<00:09,  6.09it/s]

{'eval_loss': 0.8082786202430725, 'eval_accuracy': 0.8422680412371134, 'eval_runtime': 4.5091, 'eval_samples_per_second': 215.121, 'eval_steps_per_second': 27.056, 'epoch': 2.89}


100%|██████████| 1455/1455 [04:49<00:00,  5.03it/s]

{'train_runtime': 289.4443, 'train_samples_per_second': 40.174, 'train_steps_per_second': 5.027, 'train_loss': 0.3072073579244188, 'epoch': 3.0}


TrainOutput(global_step=1455, training_loss=0.3072073579244188, metrics={'train_runtime': 289.4443, 'train_samples_per_second': 40.174, 'train_steps_per_second': 5.027, 'train_loss': 0.3072073579244188, 'epoch': 3.0})

In [11]:
predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

100%|██████████| 122/122 [00:04<00:00, 28.09it/s]

(970, 3) (970,)


In [12]:
trainer.evaluate()

100%|██████████| 122/122 [00:04<00:00, 27.84it/s]


{'eval_loss': 0.7949932217597961,
 'eval_accuracy': 0.8474226804123711,
 'eval_runtime': 4.4203,
 'eval_samples_per_second': 219.444,
 'eval_steps_per_second': 27.6,
 'epoch': 3.0}

In [13]:
trainer.save_model()

In [14]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]
model.safetensors:   0%|          | 16.4k/438M [00:00<3:26:17, 35.4kB/s]
training_args.bin: 100%|██████████| 4.54k/4.54k [00:00<00:00, 6.91kB/s] 
model.safetensors: 100%|██████████| 438M/438M [04:59<00:00, 1.46MB/s] 
Upload 2 LFS files: 100%|██████████| 2/2 [05:00<00:00, 150.16s/it]


'https://huggingface.co/carloszansavio/bert-financial-sentiment-analysis/tree/main/'